<a href="https://colab.research.google.com/github/swethamanchukonda/umbc_data690_fall2024/blob/main/fall2024/Assignment%2009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10/30/2024

In [2]:
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_rows", None)

## Define reusable functions

In [3]:
FIRST = "First Quartile (<=25%)"
SECOND = "Second Quartile (25-50%)"
THIRD = "Third Quartile (50-75%)"
FOURTH = "Fourth Quartile (>75%)"

def assign_quartile(value, x, y, z):
    if value <= x:
        quartile = FIRST
    elif value > x and value <= y:
        quartile = SECOND
    elif value > y and value <= z:
        quartile = THIRD
    else:
        quartile = FOURTH
    return quartile

## Load and Explore Data

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/wcj365/python-stats-dataviz/refs/heads/master/fall2024/data/World_Development_Indicators_(WDI).csv")

print(df.shape)
df.sample(3)

(4123, 9)


,Year,Country,GDP per capita (current US$),"Life expectancy at birth, total (years)","Population, total",Country Code,Region,Income Group,Lending Type
3956,2008,Uzbekistan,1082.286025,68.765000,27302800.0,UZB,Europe & Central Asia,Lower middle income,Blend
2216,2016,Luxembourg,106899.293550,82.685366,582014.0,LUX,Europe & Central Asia,High income,Not classified
2456,2009,Moldova,1898.439757,69.573000,2865213.0,MDA,Europe & Central Asia,Upper middle income,IBRD


In [5]:
column_names_dict = {
    "GDP per capita (current US$)" : "GDP per Capita",
    "Life expectancy at birth, total (years)" : "Life Expectancy",
    "Population, total" : "Population"
}

df = df.rename(columns=column_names_dict)
df.sample(3)

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type
776,2020,Chile,13173.784794,79.377000,19300315.0,CHL,Latin America & Caribbean,High income,IBRD
2990,2011,Portugal,23217.295497,80.470732,10557560.0,PRT,Europe & Central Asia,High income,Not classified
239,2015,Azerbaijan,5500.503646,71.451000,9649341.0,AZE,Europe & Central Asia,Upper middle income,IBRD


In [6]:
df.sample(3).T

,1845,3741,1899
Year,2006,2021,2022
Country,Jamaica,Trinidad and Tobago,Jordan
GDP per Capita,4435.571444,16056.302041,4311.000046
Life Expectancy,71.199,72.971,NaN
Population,2689660.0,1525663.0,11285869.0
Country Code,JAM,TTO,JOR
Region,Latin America & Caribbean,Latin America & Caribbean,Middle East & North Africa
Income Group,Upper middle income,High income,Lower middle income
Lending Type,IBRD,IBRD,IBRD


## Homework Question 3 (bonus, will cover next week)

step 1 - create a column "Life Expectancy Quartile" similar to the GDP per Capita Quartile above.
Step 2 - Create a column "health and wealth status" based on the following definition
- rich and healthy (1st GDP per capita quartile and 1st life expentancy quartile)
- poor and unhealthy ((4th GDP per capita quartile and 4th life expentancy quartile)
- rich and unhealthy (1st GDP per capita quartile and 4th life expentancy quartile)
- poor and healthy ((4th GDP per capita quartile and 1st life expentancy quartile)
- Other

In [28]:
import pandas as pd
import plotly.express as px

# Suppress warnings and set display options
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", None)

# Load dataset
df = pd.read_csv("https://raw.githubusercontent.com/wcj365/python-stats-dataviz/refs/heads/master/fall2024/data/World_Development_Indicators_(WDI).csv")

# Rename columns for convenience
column_names_dict = {
    "GDP per capita (current US$)" : "GDP per Capita",
    "Life expectancy at birth, total (years)" : "Life Expectancy",
    "Population, total" : "Population"
}
df = df.rename(columns=column_names_dict)

# Filter data for the year 2020 and remove rows with missing values in relevant columns
df_2020 = df[df["Year"] == 2020].dropna(subset=["GDP per Capita", "Life Expectancy"])

# Step 1: Automatically assign quartiles for Life Expectancy and GDP per Capita using pd.qcut
df_2020['Life Expectancy Quartile'] = pd.qcut(df_2020['Life Expectancy'], q=4, labels=["First", "Second", "Third", "Fourth"])
df_2020['GDP Quartile'] = pd.qcut(df_2020['GDP per Capita'], q=4, labels=["First", "Second", "Third", "Fourth"])

# Step 2: Define 'Health and Wealth Status' using a dictionary to map conditions
def determine_health_wealth_status(row):
    status_map = {
        ('First', 'First'): 'Rich and healthy',
        ('Fourth', 'Fourth'): 'Poor and unhealthy',
        ('First', 'Fourth'): 'Rich and unhealthy',
        ('Fourth', 'First'): 'Poor and healthy'
    }
    return status_map.get((row['GDP Quartile'], row['Life Expectancy Quartile']), 'Other')

# Apply the function to create 'Health and Wealth Status' column
df_2020['Health and Wealth Status'] = df_2020.apply(determine_health_wealth_status, axis=1)

# Show a sample of the data
print(df_2020[['Country', 'GDP per Capita', 'Life Expectancy', 'GDP Quartile', 'Life Expectancy Quartile', 'Health and Wealth Status']].sample(10))

# Create the plot
fig = px.choropleth(
    df_2020,
    locations="Country",
    locationmode='country names',
    color="Health and Wealth Status",
    hover_name="Country",
    title="Health and Wealth Status of Countries in 2020"
)

# Show the plot
fig.show()


                Country  GDP per Capita  Life Expectancy GDP Quartile  \
586        Burkina Faso      823.552411        59.731000        First   
814            Colombia     5304.289129        74.769000       Second   
1251      Faroe Islands    62234.968680        83.197561       Fourth   
1156  Equatorial Guinea     6198.942524        60.708000        Third   
2999           Portugal    22242.406418        80.975610       Fourth   
187               Aruba    24008.127822        75.723000       Fourth   
2619              Nauru    10124.700622        63.437000        Third   
2657        Netherlands    52162.570115        81.358537       Fourth   
16          Afghanistan      512.055098        62.575000        First   
1802        Isle of Man    79530.605484        80.588000       Fourth   

     Life Expectancy Quartile Health and Wealth Status  
586                     First         Rich and healthy  
814                     Third                    Other  
1251                   Fo

In [7]:
print(df.shape)
df.sample(5)

(4123, 9)


,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type
2593,2013,Namibia,5463.031366,58.694,2204510.0,NAM,Sub-Saharan Africa,Upper middle income,IBRD
1127,2010,El Salvador,3017.307395,71.848,6114034.0,SLV,Latin America & Caribbean,Upper middle income,IBRD
3490,2017,St. Martin (French part),NaN,78.590,34496.0,MAF,Latin America & Caribbean,High income,Not classified
1850,2011,Jamaica,5259.931753,73.094,2746169.0,JAM,Latin America & Caribbean,Upper middle income,IBRD
1096,2017,Ecuador,6246.404252,76.972,16696944.0,ECU,Latin America & Caribbean,Upper middle income,IBRD


In [8]:
df_2020 = df[df["Year"] == 2020]
print(df_2020.shape)
df_2020.sample(5)

(217, 9)


,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type
510,2020,Brazil,6923.699912,74.009000,213196304.0,BRA,Latin America & Caribbean,Upper middle income,IBRD
2676,2020,New Caledonia,34877.635635,80.834951,271080.0,NCL,East Asia & Pacific,High income,Not classified
2334,2020,Mali,822.906137,58.633000,21224040.0,MLI,Sub-Saharan Africa,Low income,IDA
2733,2020,Niger,564.841662,61.451000,24333639.0,NER,Sub-Saharan Africa,Low income,IDA
1498,2020,Grenada,8437.536782,74.924000,123663.0,GRD,Latin America & Caribbean,Upper middle income,Blend


In [9]:
df_2020 = df_2020.dropna(subset=["GDP per Capita", "Life Expectancy"])
df_2020.shape

(202, 9)

In [10]:
# Step 1: Creates 'Life Expectancy Quartile' column and lists the quartile the row falls under.
df_2020['Life Expectancy'].describe()



,Life Expectancy
count,202.000000
mean,72.314599
std,7.451962
min,52.777000
25%,66.779750
50%,72.871500
75%,77.981750
max,85.497561


In [11]:
x = df_2020['Life Expectancy'].describe()['25%']
y = df_2020['Life Expectancy'].describe()['50%']
z = df_2020['Life Expectancy'].describe()['75%']

In [12]:
df_2020['Life Expectancy Quartile'] = df_2020['Life Expectancy'].apply(assign_quartile, args=(x, y, z,))
df_2020.sample(5)

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile
3227,2020,Seychelles,12020.021064,77.236585,98462.0,SYC,Sub-Saharan Africa,High income,IBRD,Third Quartile (50-75%)
3075,2020,Russian Federation,10194.441406,71.338780,144073139.0,RUS,Europe & Central Asia,Upper middle income,IBRD,Second Quartile (25-50%)
2505,2020,Mongolia,4041.174146,72.141000,3294335.0,MNG,East Asia & Pacific,Lower middle income,IBRD,Second Quartile (25-50%)
2410,2020,Mauritius,9011.042884,74.177073,1266014.0,MUS,Sub-Saharan Africa,Upper middle income,IBRD,Third Quartile (50-75%)
947,2020,Cuba,9499.572504,77.567000,11300698.0,CUB,Latin America & Caribbean,Upper middle income,Not classified,Third Quartile (50-75%)


In [13]:
x = df_2020['GDP per Capita'].describe()['25%']
y = df_2020['GDP per Capita'].describe()['50%']
z = df_2020['GDP per Capita'].describe()['75%']

print(x, y, z)

2188.047693164725 5920.260418960404 20761.21119943435


In [14]:
# Step 2: Creates 'GDP Quartile' column and lists the quartile the row falls under.
# Next, creates 'Health and Wealth Status' column which categorizes a row if
# a country and a year falls under (4) different categories.


df_2020['GDP Quartile'] =  df_2020['GDP per Capita'].apply(assign_quartile, args=(x, y, z,))
df_2020.sample(10)


,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile
3778,2020,Turkiye,8638.739133,75.850000,83384680.0,TUR,Europe & Central Asia,Upper middle income,IBRD,Third Quartile (50-75%),Third Quartile (50-75%)
4120,2020,Zimbabwe,1372.696674,61.124000,15669666.0,ZWE,Sub-Saharan Africa,Lower middle income,Blend,First Quartile (<=25%),First Quartile (<=25%)
2657,2020,Netherlands,52162.570115,81.358537,17441500.0,NLD,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%)
2809,2020,Norway,68340.018103,83.209756,5379475.0,NOR,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%)
3930,2020,United States,63528.634303,76.980488,331511512.0,USA,North America,High income,Not classified,Third Quartile (50-75%),Fourth Quartile (>75%)
3379,2020,South Africa,5753.066494,65.252000,58801927.0,ZAF,Sub-Saharan Africa,Upper middle income,IBRD,First Quartile (<=25%),Second Quartile (25-50%)
4082,2020,"Yemen, Rep.",578.512010,64.650000,32284046.0,YEM,Middle East & North Africa,Low income,IDA,First Quartile (<=25%),First Quartile (<=25%)
3873,2020,Ukraine,3751.737305,71.185122,44132049.0,UKR,Europe & Central Asia,Lower middle income,IBRD,Second Quartile (25-50%),Second Quartile (25-50%)
3360,2020,Somalia,556.578066,55.967000,16537016.0,SOM,Sub-Saharan Africa,Low income,IDA,First Quartile (<=25%),First Quartile (<=25%)
2923,2020,Paraguay,5353.348065,73.182000,6618695.0,PRY,Latin America & Caribbean,Upper middle income,IBRD,Third Quartile (50-75%),Second Quartile (25-50%)


In [15]:
def find_status(row):
  if (row['Life Expectancy Quartile'] == FIRST) and (row['GDP Quartile'] == FIRST):
    return 'Poor and unhealthy'
  elif (row['Life Expectancy Quartile'] == FOURTH) and (row['GDP Quartile'] == FIRST):
    return 'Poor but healthy'
  elif (row['Life Expectancy Quartile'] == FOURTH) and (row['GDP Quartile'] == FOURTH):
    return 'Rich and healthy'
  elif (row['Life Expectancy Quartile'] == THIRD) and (row['GDP Quartile'] == FOURTH):
    return 'Rich and somewhat healthy'
  elif (row['Life Expectancy Quartile'] == SECOND) and (row['GDP Quartile'] == FOURTH):
    return 'Rich and somewhat unhealthy'
  elif (row['Life Expectancy Quartile'] == FIRST) and (row['GDP Quartile'] == FOURTH):
    return 'Rich but unhealthy'
  else:
    return 'Other'

df_2020['Health and Wealth Status'] = df_2020.apply(find_status,axis=1)
df_2020.sample(10)

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status
2695,2020,New Zealand,41760.594784,82.256098,5090200.0,NZL,East Asia & Pacific,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
2239,2020,"Macao SAR, China",37474.734595,85.184000,676283.0,MAC,East Asia & Pacific,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
2733,2020,Niger,564.841662,61.451000,24333639.0,NER,Sub-Saharan Africa,Low income,IDA,First Quartile (<=25%),First Quartile (<=25%),Poor and unhealthy
4101,2020,Zambia,956.831729,62.380000,18927715.0,ZMB,Sub-Saharan Africa,Lower middle income,IDA,First Quartile (<=25%),First Quartile (<=25%),Poor and unhealthy
2467,2020,Moldova,4376.242493,70.166000,2635130.0,MDA,Europe & Central Asia,Upper middle income,IBRD,Second Quartile (25-50%),Second Quartile (25-50%),Other
2372,2020,Marshall Islands,5545.600267,64.977000,43413.0,MHL,East Asia & Pacific,Upper middle income,IDA,First Quartile (<=25%),Second Quartile (25-50%),Other
4120,2020,Zimbabwe,1372.696674,61.124000,15669666.0,ZWE,Sub-Saharan Africa,Lower middle income,Blend,First Quartile (<=25%),First Quartile (<=25%),Poor and unhealthy
2087,2020,Latvia,18096.202707,75.185366,1900449.0,LVA,Europe & Central Asia,High income,Not classified,Third Quartile (50-75%),Third Quartile (50-75%),Other
681,2020,Canada,43349.677856,81.670488,38007166.0,CAN,North America,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
2581,2020,Myanmar,1479.613689,66.797000,53423198.0,MMR,East Asia & Pacific,Lower middle income,IDA,Second Quartile (25-50%),First Quartile (<=25%),Other


In [16]:
df_2020["Health and Wealth Status"].value_counts()

,count
Health and Wealth Status,
Other,113
Rich and healthy,42
Poor and unhealthy,38
Rich and somewhat healthy,7
Rich and somewhat unhealthy,2


# Countries that belong to each status category.

In [17]:
df_rich_healthy = df_2020[df_2020["Health and Wealth Status"] == "Rich and healthy"]
print(df_rich_healthy.shape)
df_rich_healthy.sample(5)

(42, 12)


,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status
1650,2020,"Hong Kong SAR, China",46109.229995,85.497561,7481000.0,HKG,East Asia & Pacific,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
225,2020,Austria,48789.497850,81.192683,8916864.0,AUT,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
1289,2020,Finland,49169.719339,81.931707,5529543.0,FIN,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
3588,2020,Switzerland,85897.784334,83.000000,8638167.0,CHE,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
3018,2020,Puerto Rico,31427.429114,78.041000,3281557.0,PRI,Latin America & Caribbean,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy


In [18]:
df_rich_healthy["Country"].unique()

array(['Australia', 'Austria', 'Bahrain', 'Belgium', 'Bermuda', 'Canada',
       'Channel Islands', 'Cyprus', 'Czechia', 'Denmark', 'Estonia',
       'Faroe Islands', 'Finland', 'France', 'Germany',
       'Hong Kong SAR, China', 'Iceland', 'Ireland', 'Isle of Man',
       'Israel', 'Italy', 'Japan', 'Korea, Rep.', 'Liechtenstein',
       'Luxembourg', 'Macao SAR, China', 'Malta', 'Netherlands',
       'New Caledonia', 'New Zealand', 'Norway', 'Portugal',
       'Puerto Rico', 'Qatar', 'Singapore', 'Slovenia', 'Spain', 'Sweden',
       'Switzerland', 'United Arab Emirates', 'United Kingdom',
       'Virgin Islands (U.S.)'], dtype=object)

In [19]:
df_poor_unhealthy = df_2020[df_2020["Health and Wealth Status"] == "Poor and unhealthy"]
print(df_poor_unhealthy.shape)
df_poor_unhealthy["Country"].unique()

(38, 12)


array(['Afghanistan', 'Angola', 'Benin', 'Burkina Faso', 'Burundi',
       'Cameroon', 'Central African Republic', 'Chad', 'Comoros',
       'Congo, Dem. Rep.', 'Congo, Rep.', 'Ethiopia', 'Gambia, The',
       'Ghana', 'Guinea', 'Guinea-Bissau', 'Haiti', 'Kenya', 'Lesotho',
       'Liberia', 'Madagascar', 'Malawi', 'Mali', 'Mauritania',
       'Mozambique', 'Niger', 'Nigeria', 'Pakistan', 'Rwanda',
       'Sierra Leone', 'Somalia', 'Sudan', 'Tanzania', 'Togo', 'Uganda',
       'Yemen, Rep.', 'Zambia', 'Zimbabwe'], dtype=object)

In [20]:
df_poor_healthy = df_2020[df_2020["Health and Wealth Status"] == "Poor but healthy"]
print(df_poor_healthy.shape)
df_poor_healthy["Country"].unique()

(0, 12)


array([], dtype=object)

In [21]:
df_rich_unhealthy = df_2020[df_2020["Health and Wealth Status"] == "Rich but unhealthy"]
print(df_rich_unhealthy.shape)
df_rich_unhealthy["Country"].unique()

(0, 12)


array([], dtype=object)

In [22]:
df_rich_unhealthy

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status


In [23]:
df_other = df_2020[df_2020["Health and Wealth Status"] == "Other"]
df_other["Country"].nunique()

113

# Choropleth - Chorom (colorful), pleth -map, place.

Plot population of countries over a map using plotly

In [24]:
# Create the plot
fig = px.choropleth(
    df_2020,
    locations="Country",
    locationmode='country names',
    color="Health and Wealth Status",
    hover_name="Country",
#    color_continuous_scale=px.colors.sequential.Inferno
)

# Show the plot
fig.show()

## Home work Question 1:

Save the Plotly interactive chart to a HTML file.

Make sure specify the option to reduce the size of the file.

In [29]:
import plotly.io as pio
fig = px.choropleth(
    df_2020,
    locations="Country",
    locationmode='country names',
    color="Health and Wealth Status",
    hover_name="Country",
#    color_continuous_scale=px.colors.sequential.Inferno
)

# Show the plot
fig.show()


pio.write_html(fig, file='choropleth_map.html', include_plotlyjs='cdn')

from google.colab import files
files.download('choropleth_map.html')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Home work question 2:

Display country names on the Choropleth. TO make the map not messy, try on a dataframe that has only limited number of countries. For example, filter your dataset by only G7 and BRICS countries.

In [30]:
import plotly.express as px
import pandas as pd

g7_countries = ["Canada", "France", "Germany", "Italy", "Japan", "United Kingdom", "United States"]
brics_countries = ["Brazil", "Russia", "India", "China", "South Africa", "Iran", "Egypt", "Ethiopia", "United Arab Emirates"]


selected_countries = g7_countries + brics_countries

df_filtered = df_2020[df_2020['Country'].isin(selected_countries)]


fig = px.choropleth(
  df_filtered,
  locations="Country",
  locationmode='country names',
  color="Health and Wealth Status",
  hover_name="Country",

)

fig.add_scattergeo(
  locations=df_filtered['Country'],
  locationmode='country names',
  text=df_filtered['Country'],
  mode='text',
  textposition='top center',
  textfont=dict(size=10),
  showlegend=False
)

# Show the plot
fig.show()


## Home work Question 3:

Explore the website - https://g7brics.streamlit.app/

